# Logistic Regression in Bioinformatics — Homework Solution Notebook

This notebook provides a **complete worked solution** (conceptual + coding) for the Logistic Regression homework.

**Topics covered**
- log-odds, odds ratios, probability computation
- sklearn logistic regression + ROC/PR
- class imbalance and thresholding
- confounding and sign flips (simulation)
- L1/L2 regularization
- statsmodels inference (SE, z, p-values, CI)
- nonlinear relationships and feature engineering

> Instructor note: You can hide solution cells or convert this into a student version by removing the answer text/outputs.

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report
)

import statsmodels.api as sm

np.random.seed(0)

## Problem 1 — Conceptual Foundations (Answer Key)

### (a) Why logistic regression over linear regression for binary outcomes?

- Linear regression can predict values **< 0 or > 1**, which are invalid probabilities.
- Logistic regression models the probability via a **sigmoid** so predictions are in \([0,1]\).
- Logistic regression assumes the **log-odds** are linear in predictors, which is more appropriate for Bernoulli outcomes.
- Error distribution assumptions differ: linear regression assumes Gaussian errors; logistic regression uses a Bernoulli likelihood.

### (b) Meaning of \(\log(\frac{p}{1-p})\)

- \(\frac{p}{1-p}\) is the **odds** of disease (probability of disease divided by probability of no disease).
- The log transforms odds to a real-valued scale (log-odds) where additive effects are natural.

### (c) If \(\beta_1 = 0.7\), interpret
1) **Log-odds:** increasing the predictor by 1 increases log-odds by 0.7.  
2) **Odds ratio:** \(e^{0.7} \approx 2.01\). Odds multiply by about **2×** for each +1 increase.  
3) **Biological meaning:** a one-unit increase in the feature is associated with roughly **doubling** the odds of disease (holding other variables constant).

### (d) Why diminishing returns near 0 and 1?

The sigmoid \(p = \frac{1}{1+e^{-z}}\) saturates near 0 and 1. Equal changes in log-odds \(z\) produce smaller probability changes when \(p\) is already close to 0 or 1.

In [ ]:
# quick numeric check for Problem 1(c)
beta1 = 0.7
np.exp(beta1)

## Problem 2 — Manual Computation (Answer Key)
Model:  \(\log(\frac{p}{1-p}) = -2 + 0.8\cdot GeneA\)

In [ ]:
import math

def sigmoid(z):
    return 1/(1+math.exp(-z))

# (a) GeneA=0
z0 = -2 + 0.8*0
p0 = sigmoid(z0)

# (b) GeneA=3
z3 = -2 + 0.8*3
p3 = sigmoid(z3)

# (c) OR for one-unit increase
OR = math.exp(0.8)

p0, p3, OR

### Answers
- (a) GeneA = 0: logit = \(-2\) → \(p \approx 0.119\)
- (b) GeneA = 3: logit = \(-2 + 2.4 = 0.4\) → \(p \approx 0.599\)
- (c) Odds ratio for +1 GeneA: \(e^{0.8} \approx 2.23\)
- (d) Interpretation: each +1 in GeneA multiplies the odds of disease by ~2.23 (holding other variables fixed).

## Problem 3 — Tiny Dataset by Hand (Answer Key)
Dataset increases GeneX as disease becomes more likely.

**(a)** Sigmoid should increase with GeneX, crossing ~0.5 around the transition region (between 3 and 4).  
**(b)** Linear regression is inappropriate because predictions could go outside [0,1] and the error model is wrong for Bernoulli outcomes.  
**(c)** The coefficient should be **positive** because higher GeneX corresponds to higher probability of disease.

## Problem 4 — Python Implementation (sklearn) (Solution)
We generate an imbalanced dataset (80/20), fit logistic regression, and evaluate with ROC and PR.

In [ ]:
# Generate synthetic data
X, y = make_classification(
    n_samples=200,
    n_features=20,
    n_informative=5,
    n_redundant=2,
    n_repeated=0,
    n_classes=2,
    weights=[0.8, 0.2],  # imbalance: 80% controls, 20% cases
    class_sep=1.0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Pipeline: standardize then logistic regression
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

clf.fit(X_train, y_train)

# (b) coefficients + intercept
coef = clf.named_steps["lr"].coef_.ravel()
intercept = clf.named_steps["lr"].intercept_[0]

coef[:8], intercept

In [ ]:
# Predicted probabilities for first 5 samples in test set
proba_test = clf.predict_proba(X_test)[:, 1]
proba_test[:5]

In [ ]:
# (c) ROC and AUC
fpr, tpr, roc_thresh = roc_curve(y_test, proba_test)
auc = roc_auc_score(y_test, proba_test)

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve (AUC = {auc:.3f})")
plt.show()

auc

In [ ]:
# (d) Precision-Recall curve and Average Precision
prec, rec, pr_thresh = precision_recall_curve(y_test, proba_test)
ap = average_precision_score(y_test, proba_test)

plt.figure()
plt.plot(rec, prec)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title(f"Precision–Recall Curve (AP = {ap:.3f})")
plt.show()

ap

### (e) Why ROC can look “good” while PR looks “bad” under imbalance?

- ROC uses **FPR**, which divides by the number of negatives (often huge). With many negatives, a model can have small FPR even with many false positives.
- PR focuses on the **positive class**: precision penalizes false positives directly.
- In rare-disease settings, PR/AP often gives a more realistic picture of clinical usefulness.

## Problem 5 — Odds Ratio Interpretation in Genomics (Answer Key)
Given a SNP genotype coded 0/1/2 with \(\beta=-1.2\)

In [ ]:
beta = -1.2
OR_1 = np.exp(beta)         # per +1 genotype
OR_2 = np.exp(beta*2)       # from 0 -> 2
OR_1, OR_2

### Answers
- (a) Odds ratio for +1 genotype: \(e^{-1.2} \approx 0.301\)
- (b) Because OR < 1, the SNP is **protective** (higher genotype count reduces odds).
- (c) From genotype 0 → 2: odds multiply by \(e^{-2.4} \approx 0.091\). That’s ~**90% lower odds** compared to genotype 0 (holding other variables constant).

## Problem 6 — Confounding (Answer Key + Simulation)

**Key idea:** If GeneA correlates with Age, and Age affects disease, then a model omitting Age can attribute Age’s effect to GeneA.

- (a) Confounding can inflate/deflate GeneA’s coefficient.
- (b) **Yes, sign flips can occur** when GeneA is positively correlated with Age but has a negative direct effect (or vice versa).
- (c) In genomics, **population stratification** is a confounding analog: ancestry correlates with genotype and disease risk.

In [ ]:
# Simulation demonstrating potential sign flip

rng = np.random.default_rng(1)
n = 2000

# Age influences disease positively
age = rng.normal(50, 10, size=n)

# GeneA correlated with age (confounding): older individuals tend to have higher GeneA
geneA = 0.08*age + rng.normal(0, 1, size=n)

# True direct effect: GeneA is mildly protective (negative), age increases risk (positive)
# logit = b0 + b_age*age + b_gene*geneA
b0, b_age, b_gene = -8.0, 0.12, -0.8

logit = b0 + b_age*age + b_gene*geneA
p = 1/(1+np.exp(-logit))
y_sim = rng.binomial(1, p, size=n)

# Fit two models with statsmodels for interpretability
df = pd.DataFrame({"y": y_sim, "GeneA": geneA, "Age": age})

X1 = sm.add_constant(df[["GeneA"]])
m1 = sm.Logit(df["y"], X1).fit(disp=False)

X2 = sm.add_constant(df[["GeneA", "Age"]])
m2 = sm.Logit(df["y"], X2).fit(disp=False)

m1.params, m2.params

If you see GeneA’s coefficient change substantially (or flip sign) after adding Age, that’s the hallmark of confounding.

## Problem 7 — Regularization (L1 vs L2 vs None) (Solution)

In [ ]:
# Compare penalties on the same train/test split
models = {
    "none (approx)": LogisticRegression(max_iter=5000, penalty="l2", C=1e6, solver="lbfgs"),
    "L2": LogisticRegression(max_iter=5000, penalty="l2", C=1.0, solver="lbfgs"),
    "L1": LogisticRegression(max_iter=5000, penalty="l1", C=1.0, solver="liblinear"),
}

coef_table = []
for name, lr in models.items():
    pipe = Pipeline([("scaler", StandardScaler()), ("lr", lr)])
    pipe.fit(X_train, y_train)
    coefs = pipe.named_steps["lr"].coef_.ravel()
    coef_table.append({
        "model": name,
        "num_near_zero(|coef|<1e-3)": int(np.sum(np.abs(coefs) < 1e-3)),
        "coef_L1_norm": float(np.sum(np.abs(coefs))),
        "coef_L2_norm": float(np.sqrt(np.sum(coefs**2))),
        "test_AUC": float(roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1])),
    })

pd.DataFrame(coef_table)

### Answers
- (a) Regularization shrinks coefficients; L1 often shrinks many to **exactly 0**.
- (b) **L1** performs feature selection (sparse coefficients).
- (c) Omics data often has \(p \gg n\) (many features, few samples). Regularization reduces overfitting and improves generalization.

## Problem 8 — Thresholding and Confusion Matrices (Solution)

In [ ]:
thresholds = [0.5, 0.3, 0.1]

rows = []
for t in thresholds:
    y_pred = (proba_test >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp+fn) else np.nan  # recall
    specificity = tn / (tn + fp) if (tn+fp) else np.nan
    precision = tp / (tp + fp) if (tp+fp) else np.nan

    rows.append({
        "threshold": t,
        "TN": tn, "FP": fp, "FN": fn, "TP": tp,
        "sensitivity(recall)": sensitivity,
        "specificity": specificity,
        "precision": precision
    })

pd.DataFrame(rows)

### Interpretation
- Lowering the threshold generally **increases sensitivity (recall)** but decreases specificity and often precision.
- (c) For **cancer screening**, you often prefer higher sensitivity (fewer missed cases), so a lower threshold (e.g., 0.3 or 0.1) may be chosen, with follow-up confirmatory testing.

## Problem 9 — Statistical Inference (statsmodels) (Solution + Answer Key)
We fit logistic regression and extract SE, z, p-values, and confidence intervals.

In [ ]:
# Fit statsmodels logistic regression on the synthetic dataset
# (using standardized predictors for stability/interpretability)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

X_train_sm = sm.add_constant(X_train_s)
X_test_sm = sm.add_constant(X_test_s)

sm_model = sm.Logit(y_train, X_train_sm).fit(disp=False)
sm_model.summary()

In [ ]:
# Extract inference quantities
params = sm_model.params
se = sm_model.bse
z = params / se
pvals = sm_model.pvalues
ci = sm_model.conf_int()
ci.columns = ["CI_low", "CI_high"]

out = pd.DataFrame({
    "coef": params,
    "std_err": se,
    "z": z,
    "p_value": pvals,
    "CI_low": ci["CI_low"],
    "CI_high": ci["CI_high"],
})
out.head(10)

### Answer Key Notes
- (c) Many logistic regression p-values are computed using the **Wald test**:
  \[
  z = \frac{\hat\beta}{SE(\hat\beta)}
  \]
  and a two-sided p-value from the standard normal distribution.
- (d) “Statistically significant but small effect size” can happen with larger samples or low noise; it means the association is reliable but may not be clinically meaningful by itself.

## Problem 10 — Nonlinearity (Answer Key + Demo)
If the true relationship is nonlinear, logistic regression with only linear terms can underfit.
We can add polynomial terms or interactions to capture curvature.

In [ ]:
# Create a simple nonlinear 1D example: disease risk increases for very low or very high x (U-shaped)
rng = np.random.default_rng(2)
n = 800
x = rng.uniform(-3, 3, size=n)

# True probability depends on x^2 (nonlinear)
logit_true = -2 + 1.2*(x**2)  # U-shaped risk
p_true = 1/(1+np.exp(-logit_true))
y_nl = rng.binomial(1, p_true)

# Fit linear logistic regression (misspecified)
X_lin = x.reshape(-1,1)
X_train_nl, X_test_nl, y_train_nl, y_test_nl = train_test_split(X_lin, y_nl, test_size=0.3, random_state=0, stratify=y_nl)

lin_lr = Pipeline([("scaler", StandardScaler()), ("lr", LogisticRegression(max_iter=2000))])
lin_lr.fit(X_train_nl, y_train_nl)
proba_lin = lin_lr.predict_proba(X_test_nl)[:,1]
auc_lin = roc_auc_score(y_test_nl, proba_lin)

# Fit logistic regression with polynomial feature x^2
X_poly = np.column_stack([x, x**2])
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_poly, y_nl, test_size=0.3, random_state=0, stratify=y_nl)

poly_lr = Pipeline([("scaler", StandardScaler()), ("lr", LogisticRegression(max_iter=2000))])
poly_lr.fit(Xp_train, yp_train)
proba_poly = poly_lr.predict_proba(Xp_test)[:,1]
auc_poly = roc_auc_score(yp_test, proba_poly)

auc_lin, auc_poly

### Answers
- (a) If true log-odds are nonlinear, a linear logit model may underfit and give biased probability estimates.
- (b) Fixes: add polynomial terms, splines, interactions, or switch to nonlinear models.
- (c) Decision trees can outperform logistic regression when relationships include strong nonlinearities and interactions, or when rules/thresholds dominate (but they can overfit without pruning/regularization).

## Bonus — Rare Disease (Answer Key)

- (a) Accuracy is not a good metric at 1% prevalence (a model that predicts all negatives gets 99% accuracy).
- (b) Prefer PR/Average Precision, Recall at fixed Precision, or cost-sensitive metrics; also consider ROC but interpret carefully.
- (c) Training adjustments: class weights, focal loss (in other models), undersampling/oversampling (SMOTE), and careful threshold selection.